In [1]:
import os
from dotenv import load_dotenv, find_dotenv
from pydantic import BaseModel
from openai import AzureOpenAI

load_dotenv(find_dotenv())

client = AzureOpenAI(
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version=os.environ.get("OPENAI_API_VERSION"),
    azure_endpoint=os.environ.get("OPENAI_API_BASE"),
    timeout=10.0,
    max_retries=10
)


class PythonPackage(BaseModel):
    name: str
    author: str

resp = client.chat.completions.create(
    model="gpt35-turbo-16k-product-dev",
    messages=[
        {
            "role": "user",
            "content": "Return the `name`, and `author` of pydantic, in a json object."
        },
    ]
)


res = PythonPackage.model_validate_json(resp.choices[0].message.content)

In [4]:
import json
json.loads(res)

TypeError: the JSON object must be str, bytes or bytearray, not PythonPackage

In [3]:
json.loads("""
```json
{
  "name": "pydantic",
  "author": "Samuel Colvin"
}
```
""")

JSONDecodeError: Expecting value: line 2 column 1 (char 1)